In [1]:
# Import libraries and modules
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)
print(np.__version__)
np.set_printoptions(threshold=np.inf)

1.15.2-dlenv_tfe
1.18.1


# Local Development

## Arguments

In [ ]:
arguments = {}
# File arguments.
arguments["train_file_pattern"] = "data/train.tfrecord"
arguments["eval_file_pattern"] = "data/eval.tfrecord"
arguments["output_dir"] = "local_trained_model"

# Training parameters.
arguments["train_batch_size"] = 32
arguments["train_steps"] = 400

# Eval parameters.
arguments["eval_batch_size"] = 32
arguments["eval_steps"] = 10
arguments["start_delay_secs"] = 600
arguments["throttle_secs"] = 600

# Image parameters.
arguments["height"] = 32
arguments["width"] = 32
arguments["depth"] = 3

# Shared parameters.
arguments["num_steps_until_growth"] = 100

# Full lists for full 1024x1024 network growth.
full_conv_num_filters = [[512, 512], [512, 512], [512, 512], [512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]
full_conv_kernel_sizes = [[4, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
full_conv_strides = [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]

# Set final image size as a multiple of 2, starting at 4.
image_size = 64
prop_list_len = max(
    min(int(math.log(image_size, 2) - 1), len(full_conv_num_filters)), 1
)

# Get slices of lists.
conv_num_filters = convert_conv_layer_property_lists_to_string(
    full_conv_num_filters, prop_list_len
)
print("conv_num_filters = {}".format(conv_num_filters))
conv_kernel_sizes = convert_conv_layer_property_lists_to_string(
    full_conv_kernel_sizes, prop_list_len
)
print("conv_kernel_sizes = {}".format(conv_kernel_sizes))
conv_strides = convert_conv_layer_property_lists_to_string(
    full_conv_strides, prop_list_len
)
print("conv_strides = {}".format(conv_strides))

arguments["conv_num_filters"] = conv_num_filters
arguments["conv_kernel_sizes"] = conv_kernel_sizes
arguments["conv_strides"] = conv_strides

# Generator parameters.
arguments["latent_size"] = 512
arguments["generator_projection_dims"] = [4, 4, 512]
arguments["generator_l1_regularization_scale"] = 0.01
arguments["generator_l2_regularization_scale"] = 0.01
arguments["generator_optimizer"] = "GradientDescent"
arguments["generator_learning_rate"] = 0.0001
arguments["generator_clip_gradients"] = 2.0
arguments["generator_train_steps"] = 1

# Discriminator hyperparameters.
arguments["discriminator_l1_regularization_scale"] = 0.01
arguments["discriminator_l2_regularization_scale"] = 0.01
arguments["discriminator_optimizer"] = "GradientDescent"
arguments["discriminator_learning_rate"] = 0.0001
arguments["discriminator_clip_gradients"] = 2.0
arguments["discriminator_gradient_penalty_coefficient"] = 10.0
arguments["discriminator_train_steps"] = 1


## print_object.py

In [ ]:
def print_obj(function_name, object_name, object_value):
    """Prints enclosing function, object name, and object value.

    Args:
        function_name: str, name of function.
        object_name: str, name of object.
        object_value: object, value of passed object.
    """
#     pass
    print("{}: {} = {}".format(function_name, object_name, object_value))


## input.py

In [ ]:
def decode_example(protos, params):
    """Decodes TFRecord file into tensors.

    Given protobufs, decode into image and label tensors.

    Args:
        protos: protobufs from TFRecord file.
        params: dict, user passed parameters.

    Returns:
        Image and label tensors.
    """
    # Create feature schema map for protos.
    features = {
        "image_raw": tf.FixedLenFeature(shape=[], dtype=tf.string),
        "label": tf.FixedLenFeature(shape=[], dtype=tf.int64)
    }

    # Parse features from tf.Example.
    parsed_features = tf.parse_single_example(
        serialized=protos, features=features
    )
    print_obj("\ndecode_example", "features", features)

    # Convert from a scalar string tensor (whose single string has
    # length height * width * depth) to a uint8 tensor with shape
    # [height * width * depth].
    image = tf.decode_raw(
        input_bytes=parsed_features["image_raw"], out_type=tf.uint8
    )
    print_obj("decode_example", "image", image)

    # Reshape flattened image back into normal dimensions.
    image = tf.reshape(
        tensor=image,
        shape=[params["height"], params["width"], params["depth"]]
    )
    print_obj("decode_example", "image", image)

    # Convert from [0, 255] -> [-1.0, 1.0] floats.
    image = tf.cast(x=image, dtype=tf.float32) * (2. / 255) - 1.0
    print_obj("decode_example", "image", image)

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(x=parsed_features["label"], dtype=tf.int32)
    print_obj("decode_example", "label", label)

    return {"image": image}, label


def read_dataset(filename, mode, batch_size, params):
    """Reads CSV time series data using tf.data, doing necessary preprocessing.

    Given filename, mode, batch size, and other parameters, read CSV dataset
    using Dataset API, apply necessary preprocessing, and return an input
    function to the Estimator API.

    Args:
        filename: str, file pattern that to read into our tf.data dataset.
        mode: The estimator ModeKeys. Can be TRAIN or EVAL.
        batch_size: int, number of examples per batch.
        params: dict, dictionary of user passed parameters.

    Returns:
        An input function.
    """
    def _input_fn():
        """Wrapper input function used by Estimator API to get data tensors.

        Returns:
            Batched dataset object of dictionary of feature tensors and label
                tensor.
        """
        # Create list of files that match pattern.
        file_list = tf.gfile.Glob(filename=filename)

        # Create dataset from file list.
        dataset = tf.data.TFRecordDataset(
            filenames=file_list, num_parallel_reads=40
        )

        # Shuffle and repeat if training with fused op.
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.apply(
                tf.contrib.data.shuffle_and_repeat(
                    buffer_size=50 * batch_size,
                    count=None  # indefinitely
                )
            )

        # Decode CSV file into a features dictionary of tensors, then batch.
        dataset = dataset.apply(
            tf.contrib.data.map_and_batch(
                map_func=lambda x: decode_example(
                    protos=x,
                    params=params
                ),
                batch_size=batch_size,
                num_parallel_calls=4
            )
        )

        # Prefetch data to improve latency.
        dataset = dataset.prefetch(buffer_size=2)

        # Create a iterator, then get batch of features from example queue.
        batched_dataset = dataset.make_one_shot_iterator().get_next()

        return batched_dataset
    return _input_fn


## generator.py

In [ ]:
def create_generator_projection_layer(regularizer, params):
    """Creates generator projection from noise latent vector.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.

    Returns:
        Latent vector projection `Dense` layer.
    """
    # Project latent vectors.
    projection_height = params["generator_projection_dims"][0]
    projection_width = params["generator_projection_dims"][1]
    projection_depth = params["generator_projection_dims"][2]

    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # shape = (
        #     cur_batch_size,
        #     projection_height * projection_width * projection_depth
        # )
        projection_layer = tf.layers.Dense(
            units=projection_height * projection_width * projection_depth,
            activation=tf.nn.leaky_relu,
            kernel_initializer="he_normal",
            kernel_regularizer=regularizer,
            name="generator_projection_layer"
        )
        print_obj(
            "create_generator_projection_layer",
            "projection_layer",
            projection_layer
        )

    return projection_layer


def build_generator_projection_layer(projection_layer, params):
    """Builds generator projection layer internals using call.

    Args:
        projection_layer: `Dense` layer for projection of noise into image.
        params: dict, user passed parameters.

    Returns:
        Latent vector projection tensor.
    """
    # Project latent vectors.
    projection_height = params["generator_projection_dims"][0]
    projection_width = params["generator_projection_dims"][1]
    projection_depth = params["generator_projection_dims"][2]

    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # shape = (
        #     cur_batch_size,
        #     projection_height * projection_width * projection_depth
        # )
        projection_tensor = projection_layer(
            inputs=tf.zeros(
                shape=[1, params["latent_size"]], dtype=tf.float32
            )
        )
        print_obj(
            "\nbuild_generator_projection_layer",
            "projection_tensor",
            projection_tensor
        )

    return projection_tensor

def use_generator_projection_layer(Z, projection_layer, params):
    """Uses projection layer to convert random noise vector into an image.

    Args:
        Z: tensor, latent vectors of shape [cur_batch_size, latent_size].
        projection_layer: `Dense` layer for projection of noise into image.
        params: dict, user passed parameters.

    Returns:
        Latent vector projection tensor.
    """
    # Project latent vectors.
    projection_height = params["generator_projection_dims"][0]
    projection_width = params["generator_projection_dims"][1]
    projection_depth = params["generator_projection_dims"][2]

    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # shape = (
        #     cur_batch_size,
        #     projection_height * projection_width * projection_depth
        # )
        projection_tensor = projection_layer(inputs=Z)
        print_obj(
            "\nuse_generator_projection_layer", "projection_tensor", projection_tensor
        )

    # Reshape projection into "image".
    # shape = (
    #     cur_batch_size,
    #     projection_height,
    #     projection_width,
    #     projection_depth
    # )
    projection_tensor_reshaped = tf.reshape(
        tensor=projection_tensor,
        shape=[-1, projection_height, projection_width, projection_depth],
        name="generator_projection_reshaped"
    )
    print_obj(
        "use_generator_projection_layer",
        "projection_tensor_reshaped",
        projection_tensor_reshaped
    )

    return projection_tensor_reshaped


def create_generator_base_conv_layer_block(regularizer, params):
    """Creates generator base conv layer block.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.

    Returns:
        List of base conv layers.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["generator_base_conv_blocks"][0]

        # Create list of base conv layers.
        base_conv_layers = [
            tf.layers.Conv2D(
                filters=conv_block[i][3],
                kernel_size=conv_block[i][0:2],
                strides=conv_block[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="generator_base_layers_conv2d_{}_{}x{}_{}_{}".format(
                    i,
                    conv_block[i][0],
                    conv_block[i][1],
                    conv_block[i][2],
                    conv_block[i][3]
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\ncreate_generator_base_conv_layer_block",
            "base_conv_layers",
            base_conv_layers
        )

    return base_conv_layers


def build_generator_base_conv_layer_block(base_conv_layers, params):
    """Builds generator base conv layer block internals using call.

    Args:
        base_conv_layers: list, the base block's conv layers.
        params: dict, user passed parameters.

    Returns:
        List of base conv tensors.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["generator_base_conv_blocks"][0]

        # Create list of base conv layers.
        base_conv_tensors = [
            base_conv_layers[i](
                inputs=tf.zeros(
                    shape=[1] + conv_block[i][0:3], dtype=tf.float32
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\nbuild_generator_base_conv_layer_block",
            "base_conv_tensors",
            base_conv_tensors
        )

    return base_conv_tensors


def create_generator_growth_layer_block(regularizer, params, block_idx):
    """Creates generator growth block.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.
        block_idx: int, the current growth block's index.

    Returns:
        List of growth block layers.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["generator_growth_conv_blocks"][block_idx]

        # Create new inner convolutional layers.
        conv_layers = [
            tf.layers.Conv2D(
                filters=conv_block[i][3],
                kernel_size=conv_block[i][0:2],
                strides=conv_block[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="generator_growth_layers_conv2d_{}_{}_{}x{}_{}_{}".format(
                    block_idx,
                    i,
                    conv_block[i][0],
                    conv_block[i][1],
                    conv_block[i][2],
                    conv_block[i][3]
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\ncreate_generator_growth_layer_block", "conv_layers", conv_layers
        )

    return conv_layers


def build_generator_growth_layer_block(conv_layers, params, block_idx):
    """Builds generator growth block internals through call.

    Args:
        conv_layers: list, the current growth block's conv layers.
        params: dict, user passed parameters.
        block_idx: int, the current growth block's index.

    Returns:
        List of growth block tensors.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["generator_growth_conv_blocks"][block_idx]

        # Create new inner convolutional layers.
        conv_tensors = [
            conv_layers[i](
                inputs=tf.zeros(
                    shape=[1] + conv_block[i][0:3], dtype=tf.float32
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\nbuild_generator_growth_layer_block",
            "conv_tensors",
            conv_tensors
        )

    return conv_tensors


def create_generator_to_rgb_layers(regularizer, params):
    """Creates generator toRGB layers of 1x1 convs.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.

    Returns:
        List of toRGB 1x1 conv layers.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get toRGB layer properties.
        to_rgb = [
            params["generator_to_rgb_layers"][i][0][:]
            for i in range(len(params["generator_to_rgb_layers"]))
        ]

        # Create list to hold toRGB 1x1 convs.
        to_rgb_conv_layers = [
            tf.layers.Conv2D(
                filters=to_rgb[i][3],
                kernel_size=to_rgb[i][0:2],
                strides=to_rgb[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="generator_to_rgb_layers_conv2d_{}_{}x{}_{}_{}".format(
                    i, to_rgb[i][0], to_rgb[i][1], to_rgb[i][2], to_rgb[i][3]
                )
            )
            for i in range(len(to_rgb))
        ]
        print_obj(
            "\ncreate_generator_to_rgb_layers",
            "to_rgb_conv_layers",
            to_rgb_conv_layers
        )

    return to_rgb_conv_layers


def build_generator_to_rgb_layers(to_rgb_conv_layers, params):
    """Builds generator toRGB layers of 1x1 convs internals through call.

    Args:
        to_rgb_conv_layers: list, toRGB conv layers.
        params: dict, user passed parameters.

    Returns:
        List of toRGB 1x1 conv tensors.
    """
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Get toRGB layer properties.
        to_rgb = [
            params["generator_to_rgb_layers"][i][0][:]
            for i in range(len(params["generator_to_rgb_layers"]))
        ]

        # Create list to hold toRGB 1x1 convs.
        to_rgb_conv_tensors = [
            to_rgb_conv_layers[i](
                inputs=tf.zeros(shape=[1] + to_rgb[i][0:3], dtype=tf.float32))
            for i in range(len(to_rgb))
        ]
        print_obj(
            "\nbuild_generator_to_rgb_layers",
            "to_rgb_conv_tensors",
            to_rgb_conv_tensors
        )

    return to_rgb_conv_tensors


def upsample_generator_image(image, original_image_size, block_idx):
    """Upsamples generator image.

    Args:
        image: tensor, image created by generator conv block.
        original_image_size: list, the height and width dimensions of the
            original image before any growth.
        block_idx: int, index of the current generator growth block.

    Returns:
        Upsampled image tensor.
    """
    # Upsample from s X s to 2s X 2s image.
    upsampled_image = tf.image.resize(
        images=image,
        size=tf.convert_to_tensor(
            value=original_image_size,
            dtype=tf.int32,
            name="upsample_generator_image_original_image_size"
        ) * 2 ** block_idx,
        method="nearest",
        name="generator_growth_upsampled_image_{}_{}x{}_{}x{}".format(
            block_idx,
            original_image_size[0] * 2 ** (block_idx - 1),
            original_image_size[1] * 2 ** (block_idx - 1),
            original_image_size[0] * 2 ** block_idx,
            original_image_size[1] * 2 ** block_idx
        )
    )
    print_obj(
        "\nupsample_generator_image",
        "upsampled_image",
        upsampled_image
    )

    return upsampled_image


def create_base_generator_network(
        Z, projection_layer, to_rgb_conv_layers, blocks, params):
    """Creates base generator network.

    Args:
        Z: tensor, latent vectors of shape [cur_batch_size, latent_size].
        projection_layer: `Dense` layer for projection of noise into image.
        to_rgb_conv_layers: list, toRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        params: dict, user passed parameters.

    Returns:
        Final network block conv tensor.
    """
    print_obj("\ncreate_base_generator_network", "Z", Z)
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Project latent noise vectors into image.
        projection = use_generator_projection_layer(
            Z=Z, projection_layer=projection_layer, params=params
        )
        print_obj("create_base_generator_network", "projection", projection)

        # Only need the first block and toRGB conv layer for base network.
        block_layers = blocks[0]
        to_rgb_conv_layer = to_rgb_conv_layers[0]

        # Pass inputs through layer chain.
        block_conv = block_layers[0](inputs=projection)
        print_obj("create_base_generator_network", "block_conv_0", block_conv)

        for i in range(1, len(block_layers)):
            block_conv = block_layers[i](inputs=block_conv)
            print_obj(
                "create_base_generator_network",
                "block_conv_{}".format(i),
                block_conv
            )
        to_rgb_conv = to_rgb_conv_layer(inputs=block_conv)
        print_obj("create_base_generator_network", "to_rgb_conv", to_rgb_conv)

    return to_rgb_conv


def create_growth_transition_generator_network(
        Z,
        projection_layer,
        to_rgb_conv_layers,
        blocks,
        original_image_size,
        alpha_var,
        params,
        trans_idx):
    """Creates base generator network.

    Args:
        Z: tensor, latent vectors of shape [cur_batch_size, latent_size].
        projection_layer: `Dense` layer for projection of noise into image.
        to_rgb_conv_layers: list, toRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        original_image_size: list, the height and width dimensions of the
            original image before any growth.
        alpha_var: variable, alpha for weighted sum of fade-in of layers.
        params: dict, user passed parameters.
        trans_idx: int, index of current growth transition.

    Returns:
        Final network block conv tensor.
    """
    print_obj(
        "\nEntered create_growth_transition_generator_network",
        "trans_idx",
        trans_idx
    )
    print_obj("create_growth_transition_generator_network", "Z", Z)
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Project latent noise vectors into image.
        projection = use_generator_projection_layer(
            Z=Z, projection_layer=projection_layer, params=params
        )
        print_obj(
            "create_growth_transition_generator_network",
            "projection",
            projection
        )

        # Permanent blocks.
        permanent_blocks = blocks[0:trans_idx + 1]

        # Base block doesn't need any upsampling so it's handled differently.
        base_block_conv_layers = permanent_blocks[0]

        # Pass inputs through layer chain.
        block_conv = base_block_conv_layers[0](inputs=projection)
        print_obj(
            "create_growth_transition_generator_network",
            "base_block_conv_{}_0".format(trans_idx),
            block_conv
        )
        for i in range(1, len(base_block_conv_layers)):
            block_conv = base_block_conv_layers[i](inputs=block_conv)
            print_obj(
                "create_growth_transition_generator_network",
                "base_block_conv_{}_{}".format(trans_idx, i),
                block_conv
            )

        # Growth blocks require first the prev conv layer's image upsampled.
        for i in range(1, len(permanent_blocks)):
            # Upsample previous block's image.
            block_conv = upsample_generator_image(
                image=block_conv,
                original_image_size=original_image_size,
                block_idx=i
            )
            print_obj(
                "create_growth_transition_generator_network",
                "upsample_generator_image_block_conv_{}_{}".format(
                    trans_idx, i
                ),
                block_conv
            )

            block_conv_layers = permanent_blocks[i]
            for j in range(0, len(block_conv_layers)):
                block_conv = block_conv_layers[j](inputs=block_conv)
                print_obj(
                    "create_growth_transition_generator_network",
                    "block_conv_{}_{}_{}".format(trans_idx, i, j),
                    block_conv
                )

        # Upsample most recent block conv image for both side chains.
        upsampled_block_conv = upsample_generator_image(
            image=block_conv,
            original_image_size=original_image_size,
            block_idx=len(permanent_blocks)
        )
        print_obj(
            "create_growth_transition_generator_network",
            "upsampled_block_conv_{}".format(trans_idx),
            upsampled_block_conv
        )

        # Growing side chain.
        growing_block_layers = blocks[trans_idx + 1]
        growing_to_rgb_conv_layer = to_rgb_conv_layers[trans_idx + 1]

        # Pass inputs through layer chain.
        block_conv = growing_block_layers[0](inputs=upsampled_block_conv)
        print_obj(
            "create_growth_transition_generator_network",
            "growing_block_conv_{}_0".format(trans_idx),
            block_conv
        )
        for i in range(1, len(growing_block_layers)):
            block_conv = growing_block_layers[i](inputs=block_conv)
            print_obj(
                "create_growth_transition_generator_network",
                "growing_block_conv_{}_{}".format(trans_idx, i),
                block_conv
            )
        growing_to_rgb_conv = growing_to_rgb_conv_layer(inputs=block_conv)
        print_obj(
            "create_growth_transition_generator_network",
            "growing_to_rgb_conv_{}".format(trans_idx),
            growing_to_rgb_conv
        )

        # Shrinking side chain.
        shrinking_to_rgb_conv_layer = to_rgb_conv_layers[trans_idx]

        # Pass inputs through layer chain.
        shrinking_to_rgb_conv = shrinking_to_rgb_conv_layer(
            inputs=upsampled_block_conv
        )
        print_obj(
            "create_growth_transition_generator_network",
            "shrinking_to_rgb_conv_{}".format(trans_idx),
            shrinking_to_rgb_conv
        )

        # Weighted sum.
        weighted_sum = tf.add(
            x=growing_to_rgb_conv * alpha_var,
            y=shrinking_to_rgb_conv * (1.0 - alpha_var),
            name="growth_transition_weighted_sum_{}".format(trans_idx)
        )
        print_obj(
            "create_growth_transition_generator_network",
            "weighted_sum_{}".format(trans_idx),
            weighted_sum
        )

    return weighted_sum


def create_final_generator_network(
        Z, projection_layer, to_rgb_conv_layers, blocks, original_image_size, params):
    """Creates base generator network.

    Args:
        Z: tensor, latent vectors of shape [cur_batch_size, latent_size].
        projection_layer: `Dense` layer for projection of noise into image.
        to_rgb_conv_layers: list, toRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        original_image_size: list, the height and width dimensions of the
            original image before any growth.
        params: dict, user passed parameters.

    Returns:
        Final network block conv tensor.
    """
    print_obj("\ncreate_final_generator_network", "Z", Z)
    with tf.variable_scope(name_or_scope="generator", reuse=tf.AUTO_REUSE):
        # Project latent noise vectors into image.
        projection = use_generator_projection_layer(
            Z=Z, projection_layer=projection_layer, params=params
        )
        print_obj("create_final_generator_network", "projection", projection)

        # Base block doesn't need any upsampling so it's handled differently.
        base_block_conv_layers = blocks[0]

        # Pass inputs through layer chain.
        block_conv = base_block_conv_layers[0](inputs=projection)
        print_obj(
            "\ncreate_final_generator_network",
            "base_block_conv",
            block_conv
        )

        for i in range(1, len(base_block_conv_layers)):
            block_conv = base_block_conv_layers[i](inputs=block_conv)
            print_obj(
                "create_final_generator_network",
                "base_block_conv_{}".format(i),
                block_conv
            )

        # Growth blocks require first the prev conv layer's image upsampled.
        for i in range(1, len(blocks)):
            # Upsample previous block's image.
            block_conv = upsample_generator_image(
                image=block_conv,
                original_image_size=original_image_size,
                block_idx=i
            )
            print_obj(
                "create_final_generator_network",
                "upsample_generator_image_block_conv_{}".format(i),
                block_conv
            )

            block_conv_layers = blocks[i]
            for j in range(0, len(block_conv_layers)):
                block_conv = block_conv_layers[j](inputs=block_conv)
                print_obj(
                    "create_final_generator_network",
                    "block_conv_{}_{}".format(i, j),
                    block_conv
                )

        # Only need the last toRGB conv layer.
        to_rgb_conv_layer = to_rgb_conv_layers[-1]

        # Pass inputs through layer chain.
        to_rgb_conv = to_rgb_conv_layer(inputs=block_conv)
        print_obj(
            "create_final_generator_network", "to_rgb_conv", to_rgb_conv
        )

    return to_rgb_conv


def generator_network(Z, alpha_var, params):
    """Creates generator network and returns generated output.

    Args:
        Z: tensor, latent vectors of shape [cur_batch_size, latent_size].
        alpha_var: variable, alpha for weighted sum of fade-in of layers.
        params: dict, user passed parameters.

    Returns:
        Generated outputs tensor of shape
            [cur_batch_size, height * width * depth].
    """
    print_obj("\ngenerator_network", "Z", Z)

    # Create regularizer for layer kernel weights.
    regularizer = tf.contrib.layers.l1_l2_regularizer(
        scale_l1=params["generator_l1_regularization_scale"],
        scale_l2=params["generator_l2_regularization_scale"]
    )

    # Create projection dense layer to turn random noise vector into image.
    projection_layer = create_generator_projection_layer(
        regularizer=regularizer, params=params
    )

    # Build projection layer internals using call.
    projection_tensor = build_generator_projection_layer(
        projection_layer=projection_layer, params=params
    )

    with tf.control_dependencies(control_inputs=[projection_tensor]):
        # Create empty lists to hold generator convolutional layer/tensor blocks.
        block_layers = []
        block_tensors = []

        # Create base convolutional layers, for post-growth.
        block_layers.append(
            create_generator_base_conv_layer_block(
                regularizer=regularizer, params=params
            )
        )

        # Build base convolutional layer block's internals using call.
        block_tensors.append(
            build_generator_base_conv_layer_block(
                base_conv_layers=block_layers[0], params=params
            )
        )

        # Create growth block layers.
        for block_idx in range(len(params["generator_growth_conv_blocks"])):
            block_layers.append(
                create_generator_growth_layer_block(
                    regularizer=regularizer, params=params, block_idx=block_idx
                )
            )
        print_obj("generator_network", "block_layers", block_layers)

        # Build growth block layer internals through call.
        for block_idx in range(len(params["generator_growth_conv_blocks"])):
            block_tensors.append(
                build_generator_growth_layer_block(
                    conv_layers=block_layers[block_idx + 1],
                    params=params,
                    block_idx=block_idx
                )
            )

        # Flatten block tensor lists of lists into list.
        block_tensors = [item for sublist in block_tensors for item in sublist]
        print_obj("generator_network", "block_tensors", block_tensors)

        with tf.control_dependencies(control_inputs=block_tensors):
            # Create toRGB 1x1 conv layers.
            to_rgb_conv_layers = create_generator_to_rgb_layers(
                regularizer=regularizer, params=params
            )
            print_obj(
                "generator_network", "to_rgb_conv_layers", to_rgb_conv_layers
            )

            # Build toRGB 1x1 conv layer internals through call.
            to_rgb_conv_tensors = build_generator_to_rgb_layers(
                to_rgb_conv_layers=to_rgb_conv_layers, params=params
            )
            print_obj(
                "generator_network", "to_rgb_conv_tensors", to_rgb_conv_tensors
            )

            with tf.control_dependencies(control_inputs=to_rgb_conv_tensors):
                # Get generator's output image tensor.
                train_steps = params["train_steps"]
                num_steps_until_growth = params["num_steps_until_growth"]
                num_stages = train_steps // num_steps_until_growth
                if (num_stages <= 0 or len(params["conv_num_filters"]) == 1):
                    print(
                        "\ngenerator_network: NOT GOING TO GROW, SKIP SWITCH CASE!"
                    )
                    # If never going to grow, no sense using the switch case.
                    # 4x4
                    generated_outputs = create_base_generator_network(
                        Z=Z,
                        projection_layer=projection_layer,
                        to_rgb_conv_layers=to_rgb_conv_layers,
                        blocks=block_layers,
                        params=params
                    )
                else:
                    # Find growth index based on global step and growth frequency.
                    growth_index = tf.cast(
                        x=tf.floordiv(
                            x=tf.train.get_or_create_global_step(),
                            y=params["num_steps_until_growth"],
                            name="generator_global_step_floordiv"
                        ),
                        dtype=tf.int32,
                        name="generator_growth_index"
                    )

                    # Get original image size to use for setting image shape.
                    original_image_size = params["generator_projection_dims"][0:2]

                    # Switch to case based on number of steps for gen outputs.
                    generated_outputs = tf.switch_case(
                        branch_index=growth_index,
                        branch_fns=[
                            # 4x4
                            lambda: create_base_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                params=params
                            ),
                            # 8x8
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=0
                            ),
                            # 16x16
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=1
                            ),
                            # 32x32
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=2
                            ),
                            # 64x64
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=3
                            ),
                            # 128x128
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=4
                            ),
                            # 256x256
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=5
                            ),
                            # 512x512
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=6
                            ),
                            # 1024x1024
                            lambda: create_growth_transition_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=7
                            ),
                            # 1024x1024
                            lambda: create_final_generator_network(
                                Z=Z,
                                projection_layer=projection_layer,
                                to_rgb_conv_layers=to_rgb_conv_layers,
                                blocks=block_layers,
                                original_image_size=original_image_size,
                                params=params
                            )
                        ],
                        name="generator_switch_case_generated_outputs"
                    )

                print_obj(
                    "generator_network", "generated_outputs", generated_outputs
                )

    return generated_outputs


def get_generator_loss(fake_logits, params):
    """Gets generator loss.

    Args:
        fake_logits: tensor, shape of [cur_batch_size, 1] that came from
            discriminator having processed generator's output image.
        params: dict, user passed parameters.

    Returns:
        Tensor of generator's total loss of shape [].
    """
    # Calculate base generator loss.
    generator_loss = -tf.reduce_mean(
        input_tensor=fake_logits,
        name="generator_loss"
    )
    print_obj("\nget_generator_loss", "generator_loss", generator_loss)

    # Get generator regularization losses.
    generator_reg_loss = regularization.get_regularization_loss(
        params=params, scope="generator"
    )
    print_obj(
        "get_generator_loss",
        "generator_reg_loss",
        generator_reg_loss
    )

    # Combine losses for total losses.
    generator_total_loss = tf.math.add(
        x=generator_loss,
        y=generator_reg_loss,
        name="generator_total_loss"
    )
    print_obj(
        "get_generator_loss", "generator_total_loss", generator_total_loss
    )

    return generator_total_loss


## discriminator.py

In [ ]:
def create_discriminator_from_rgb_layers(regularizer, params):
    """Creates discriminator fromRGB layers of 1x1 convs.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.

    Returns:
        List of fromRGB 1x1 conv layers.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get fromRGB layer properties.
        from_rgb = [
            params["discriminator_from_rgb_layers"][i][0][:]
            for i in range(len(params["discriminator_from_rgb_layers"]))
        ]

        # Create list to hold toRGB 1x1 convs.
        from_rgb_conv_layers = [
            tf.layers.Conv2D(
                filters=from_rgb[i][3],
                kernel_size=from_rgb[i][0:2],
                strides=from_rgb[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="discriminator_from_rgb_layers_conv2d_{}_{}x{}_{}_{}".format(
                    i,
                    from_rgb[i][0],
                    from_rgb[i][1],
                    from_rgb[i][2],
                    from_rgb[i][3]
                )
            )
            for i in range(len(from_rgb))
        ]
        print_obj(
            "\ncreate_discriminator_from_rgb_layers",
            "from_rgb_conv_layers",
            from_rgb_conv_layers
        )

    return from_rgb_conv_layers


def build_discriminator_from_rgb_layers(from_rgb_conv_layers, params):
    """Creates discriminator fromRGB layers of 1x1 convs.

    Args:
        from_rgb_conv_layers: list, fromGRB con layers.
        params: dict, user passed parameters.

    Returns:
        List of fromRGB 1x1 conv tensors.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get fromRGB layer properties.
        from_rgb = [
            params["discriminator_from_rgb_layers"][i][0][:]
            for i in range(len(params["discriminator_from_rgb_layers"]))
        ]

        # Create list to hold toRGB 1x1 convs.
        from_rgb_conv_tensors = [
            from_rgb_conv_layers[i](
                inputs=tf.zeros(
                    shape=[1] + from_rgb[i][0:3], dtype=tf.float32
                )
            )
            for i in range(len(from_rgb))
        ]
        print_obj(
            "\nbuild_discriminator_from_rgb_layers",
            "from_rgb_conv_tensors",
            from_rgb_conv_tensors
        )

    return from_rgb_conv_tensors


def create_discriminator_base_conv_layer_block(regularizer, params):
    """Creates discriminator base conv layer block.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.

    Returns:
        List of base conv layers.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["discriminator_base_conv_blocks"][0]

        # Create list of base conv layers.
        base_conv_layers = [
            tf.layers.Conv2D(
                filters=conv_block[i][3],
                kernel_size=conv_block[i][0:2],
                strides=conv_block[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="discriminator_base_layers_conv2d_{}_{}x{}_{}_{}".format(
                    i,
                    conv_block[i][0],
                    conv_block[i][1],
                    conv_block[i][2],
                    conv_block[i][3]
                )
            )
            for i in range(len(conv_block) - 1)
        ]

        # Have valid padding for layer just before flatten and logits.
        base_conv_layers.append(
            tf.layers.Conv2D(
                filters=conv_block[-1][3],
                kernel_size=conv_block[-1][0:2],
                strides=conv_block[-1][4:6],
                padding="valid",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="discriminator_base_layers_conv2d_{}_{}x{}_{}_{}".format(
                    len(conv_block) - 1,
                    conv_block[-1][0],
                    conv_block[-1][1],
                    conv_block[-1][2],
                    conv_block[-1][3]
                )
            )
        )
        print_obj(
            "\ncreate_discriminator_base_conv_layer_block",
            "base_conv_layers",
            base_conv_layers
        )

    return base_conv_layers


def build_discriminator_base_conv_layer_block(base_conv_layers, params):
    """Creates discriminator base conv layer block.

    Args:
        base_conv_layers: list, base conv block's layers.
        params: dict, user passed parameters.

    Returns:
        List of base conv tensors.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["discriminator_base_conv_blocks"][0]

        # Create list of base conv layer tensors.
        base_conv_tensors = [
            base_conv_layers[i](
                inputs=tf.zeros(
                    shape=[1] + conv_block[i][0:3], dtype=tf.float32
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\nbase_conv_layers",
            "base_conv_tensors",
            base_conv_tensors
        )

    return base_conv_tensors


def create_discriminator_growth_layer_block(regularizer, params, block_idx):
    """Creates discriminator growth block.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.
        params: dict, user passed parameters.
        block_idx: int, the current growth block's index.

    Returns:
        List of growth block layers.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["discriminator_growth_conv_blocks"][block_idx]

        # Create new inner convolutional layers.
        conv_layers = [
            tf.layers.Conv2D(
                filters=conv_block[i][3],
                kernel_size=conv_block[i][0:2],
                strides=conv_block[i][4:6],
                padding="same",
                activation=tf.nn.leaky_relu,
                kernel_initializer="he_normal",
                kernel_regularizer=regularizer,
                name="discriminator_growth_layers_conv2d_{}_{}_{}x{}_{}_{}".format(
                    block_idx,
                    i,
                    conv_block[i][0],
                    conv_block[i][1],
                    conv_block[i][2],
                    conv_block[i][3]
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\ncreate_discriminator_growth_layer_block",
            "conv_layers",
            conv_layers
        )

        # Down sample from 2s X 2s to s X s image.
        downsampled_image_layer = tf.layers.AveragePooling2D(
            pool_size=(2, 2),
            strides=(2, 2),
            name="discriminator_growth_downsampled_image_{}".format(
                block_idx
            )
        )
        print_obj(
            "create_discriminator_growth_layer_block",
            "downsampled_image_layer",
            downsampled_image_layer
        )

    return conv_layers + [downsampled_image_layer]


def build_discriminator_growth_layer_block(conv_layers, params, block_idx):
    """Creates discriminator growth block.

    Args:
        list, the current growth block's conv layers.
        params: dict, user passed parameters.
        block_idx: int, the current growth block's index.

    Returns:
        List of growth block layers.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Get conv block layer properties.
        conv_block = params["discriminator_growth_conv_blocks"][block_idx]

        # Create new inner convolutional layers.
        conv_tensors = [
            conv_layers[i](
                inputs=tf.zeros(
                    shape=[1] + conv_block[i][0:3], dtype=tf.float32
                )
            )
            for i in range(len(conv_block))
        ]
        print_obj(
            "\nbuild_discriminator_growth_layer_block",
            "conv_tensors",
            conv_tensors
        )

        # Down sample from 2s X 2s to s X s image.
        downsampled_image_tensor = tf.layers.AveragePooling2D(
            pool_size=(2, 2),
            strides=(2, 2),
            name="discriminator_growth_downsampled_image_{}".format(
                block_idx
            )
        )(inputs=conv_tensors[-1])
        print_obj(
            "build_discriminator_growth_layer_block",
            "downsampled_image_tensor",
            downsampled_image_tensor
        )

    return conv_tensors + [downsampled_image_tensor]


def create_discriminator_growth_transition_downsample_layers(params):
    """Creates discriminator growth transition downsample layers.

    Args:
        params: dict, user passed parameters.

    Returns:
        List of growth transition downsample layers.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Down sample from 2s X 2s to s X s image.
        downsample_layers = [
            tf.layers.AveragePooling2D(
                pool_size=(2, 2),
                strides=(2, 2),
                name="discriminator_growth_transition_downsample_layer_{}".format(
                    layer_idx
                )
            )
            for layer_idx in range(
                1 + len(params["discriminator_growth_conv_blocks"])
            )
        ]
        print_obj(
            "\ncreate_discriminator_growth_transition_downsample_layers",
            "downsample_layers",
            downsample_layers
        )

    return downsample_layers


def create_discriminator_logits_layer(regularizer):
    """Creates discriminator flatten and logits layer.

    Args:
        regularizer: `l1_l2_regularizer` object, regularizar for kernel
            variables.

    Returns:
        Flatten and logits layers of discriminator.
    """
    with tf.variable_scope(
        name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Flatten layer to get final block conv tensor ready for dense layer.
        flatten_layer = tf.layers.Flatten(name="discriminator_flatten_layer")
        print_obj(
            "\ncreate_discriminator_logits_layer",
            "flatten_layer",
            flatten_layer
        )

        # Final linear layer for logits.
        logits_layer = tf.layers.Dense(
            units=1,
            activation=None,
            kernel_regularizer=regularizer,
            name="discriminator_layers_dense_logits"
        )
        print_obj(
            "create_growth_transition_discriminator_network",
            "logits_layer",
            logits_layer
        )

    return flatten_layer, logits_layer


def build_discriminator_logits_layer(flatten_layer, logits_layer, params):
    """Builds flatten and logits layer internals using call.

    Args:
        flatten_layer: `Flatten` layer.
        logits_layer: `Dense` layer for logits.
        params: dict, user passed parameters.

    Returns:
        Final logits tensor of discriminator.
    """
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        block_conv_size = params["discriminator_base_conv_blocks"][-1][-1][3]

        # Flatten final block conv tensor.
        block_conv_flat = flatten_layer(
            inputs=tf.zeros(
                shape=[1, 1, 1, block_conv_size],
                dtype=tf.float32
            )
        )
        print_obj(
            "build_discriminator_logits_layer",
            "block_conv_flat",
            block_conv_flat
        )

        # Final linear layer for logits.
        logits = logits_layer(inputs=block_conv_flat)
        print_obj("build_discriminator_logits_layer", "logits", logits)

    return logits


def use_discriminator_logits_layer(
        block_conv, flatten_layer, logits_layer, params):
    """Uses flatten and logits layers to get logits tensor.

    Args:
        block_conv: tensor, output of last conv layer of discriminator.
        flatten_layer: `Flatten` layer.
        logits_layer: `Dense` layer for logits.
        params: dict, user passed parameters.

    Returns:
        Final logits tensor of discriminator.
    """
    print_obj("\nuse_discriminator_logits_layer", "block_conv", block_conv)
    # Set shape to remove ambiguity for dense layer.
    block_conv.set_shape(
        [
            block_conv.get_shape()[0],
            params["generator_projection_dims"][0] / 4,
            params["generator_projection_dims"][1] / 4,
            block_conv.get_shape()[-1]]
    )
    print_obj("use_discriminator_logits_layer", "block_conv", block_conv)

    with tf.variable_scope(name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Flatten final block conv tensor.
        block_conv_flat = flatten_layer(inputs=block_conv)
        print_obj(
            "use_discriminator_logits_layer",
            "block_conv_flat",
            block_conv_flat
        )

        # Final linear layer for logits.
        logits = logits_layer(inputs=block_conv_flat)
        print_obj("use_discriminator_logits_layer", "logits", logits)

    return logits


def create_base_discriminator_network(
        X, from_rgb_conv_layers, blocks, flatten_layer, logits_layer, params):
    """Creates base discriminator network.

    Args:
        X: tensor, input image to discriminator.
        from_rgb_conv_layers: list, fromRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        flatten_layer: `Flatten` layer.
        logits_layer: `Dense` layer for logits.
        params: dict, user passed parameters.

    Returns:
        Final logits tensor of discriminator.
    """
    print_obj("\ncreate_base_discriminator_network", "X", X)
    with tf.variable_scope(name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Only need the first fromRGB conv layer and block for base network.
        from_rgb_conv_layer = from_rgb_conv_layers[0]
        block_layers = blocks[0]

        # Pass inputs through layer chain.
        from_rgb_conv = from_rgb_conv_layer(inputs=X)
        print_obj(
            "create_base_discriminator_network",
            "from_rgb_conv",
            from_rgb_conv
        )

        block_conv = from_rgb_conv
        for i in range(len(block_layers)):
            block_conv = block_layers[i](inputs=block_conv)
            print_obj(
                "create_base_discriminator_network", "block_conv", block_conv
            )

        # Get logits now.
        logits = use_discriminator_logits_layer(
            block_conv=block_conv,
            flatten_layer=flatten_layer,
            logits_layer=logits_layer,
            params=params
        )
        print_obj("create_base_discriminator_network", "logits", logits)

    return logits


def create_growth_transition_discriminator_network(
        X,
        from_rgb_conv_layers,
        blocks,
        transition_downsample_layers,
        flatten_layer,
        logits_layer,
        alpha_var,
        params,
        trans_idx):
    """Creates base discriminator network.

    Args:
        X: tensor, input image to discriminator.
        from_rgb_conv_layers: list, fromRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        transition_downsample_layers: list, downsample layers for transition.
        flatten_layer: `Flatten` layer.
        logits_layer: `Dense` layer for logits.
        alpha_var: variable, alpha for weighted sum of fade-in of layers.
        params: dict, user passed parameters.
        trans_idx: int, index of current growth transition.

    Returns:
        Final logits tensor of discriminator.
    """
    print_obj(
        "\nEntered create_growth_transition_discriminator_network",
        "trans_idx",
        trans_idx
    )
    print_obj("create_growth_transition_discriminator_network", "X", X)
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Growing side chain.
        growing_from_rgb_conv_layer = from_rgb_conv_layers[trans_idx + 1]
        growing_block_layers = blocks[trans_idx + 1]

        # Pass inputs through layer chain.
        growing_block_conv = growing_from_rgb_conv_layer(inputs=X)
        print_obj(
            "\ncreate_growth_transition_discriminator_network",
            "growing_block_conv",
            growing_block_conv
        )
        for i in range(len(growing_block_layers)):
            growing_block_conv = growing_block_layers[i](
                inputs=growing_block_conv
            )
            print_obj(
                "create_growth_transition_discriminator_network",
                "growing_block_conv",
                growing_block_conv
            )

        # Shrinking side chain.
        transition_downsample_layer = transition_downsample_layers[trans_idx]
        shrinking_from_rgb_conv_layer = from_rgb_conv_layers[trans_idx]

        # Pass inputs through layer chain.
        transition_downsample = transition_downsample_layer(inputs=X)
        print_obj(
            "create_growth_transition_discriminator_network",
            "transition_downsample",
            transition_downsample
        )
        shrinking_from_rgb_conv = shrinking_from_rgb_conv_layer(
            inputs=transition_downsample
        )
        print_obj(
            "create_growth_transition_discriminator_network",
            "shrinking_from_rgb_conv",
            shrinking_from_rgb_conv
        )

        # Weighted sum.
        weighted_sum = tf.add(
            x=growing_block_conv * alpha_var,
            y=shrinking_from_rgb_conv * (1.0 - alpha_var),
            name="growth_transition_weighted_sum_{}".format(trans_idx)
        )
        print_obj(
            "create_growth_transition_discriminator_network",
            "weighted_sum",
            weighted_sum
        )

        # Permanent blocks.
        permanent_blocks = blocks[0:trans_idx + 1]

        # Reverse order of blocks and flatten.
        permanent_block_layers = [
            item for sublist in permanent_blocks[::-1] for item in sublist
        ]

        # Pass inputs through layer chain.
        block_conv = weighted_sum
        for i in range(len(permanent_block_layers)):
            block_conv = permanent_block_layers[i](inputs=block_conv)
            print_obj(
                "create_growth_transition_discriminator_network",
                "block_conv",
                block_conv
            )

        # Get logits now.
        logits = use_discriminator_logits_layer(
            block_conv, flatten_layer, logits_layer, params
        )
        print_obj(
            "create_growth_transition_discriminator_network", "logits", logits
        )

    return logits


def create_final_discriminator_network(
        X, from_rgb_conv_layers, blocks, flatten_layer, logits_layer, params):
    """Creates base discriminator network.

    Args:
        X: tensor, input image to discriminator.
        from_rgb_conv_layers: list, fromRGB 1x1 conv layers.
        blocks: list, lists of block layers for each block.
        flatten_layer: `Flatten` layer.
        logits_layer: `Dense` layer for logits.
        params: dict, user passed parameters.

    Returns:
        Final logits tensor of discriminator.
    """
    print_obj("\ncreate_final_discriminator_network", "X", X)
    with tf.variable_scope(
            name_or_scope="discriminator", reuse=tf.AUTO_REUSE):
        # Only need the last fromRGB conv layer.
        from_rgb_conv_layer = from_rgb_conv_layers[-1]

        # Reverse order of blocks and flatten.
        block_layers = [item for sublist in blocks[::-1] for item in sublist]

        # Pass inputs through layer chain.
        block_conv = from_rgb_conv_layer(inputs=X)
        print_obj(
            "\ncreate_final_discriminator_network",
            "block_conv",
            block_conv
        )

        for i in range(len(block_layers)):
            block_conv = block_layers[i](inputs=block_conv)
            print_obj(
                "create_final_discriminator_network", "block_conv", block_conv
            )

        # Get logits now.
        logits = use_discriminator_logits_layer(
            block_conv=block_conv,
            flatten_layer=flatten_layer,
            logits_layer=logits_layer,
            params=params
        )
        print_obj("create_final_discriminator_network", "logits", logits)

    return logits


def discriminator_network(X, alpha_var, params):
    """Creates discriminator network and returns logits.

    Args:
        X: tensor, image tensors of shape
            [cur_batch_size, height, width, depth].
        alpha_var: variable, alpha for weighted sum of fade-in of layers.
        params: dict, user passed parameters.

    Returns:
        Logits tensor of shape [cur_batch_size, 1].
    """
    print_obj("\ndiscriminator_network", "X", X)

    # Create regularizer for layer kernel weights.
    regularizer = tf.contrib.layers.l1_l2_regularizer(
        scale_l1=params["discriminator_l1_regularization_scale"],
        scale_l2=params["discriminator_l2_regularization_scale"]
    )

    # Create fromRGB 1x1 conv layers.
    from_rgb_conv_layers = create_discriminator_from_rgb_layers(
        regularizer=regularizer, params=params
    )
    print_obj(
        "discriminator_network",
        "from_rgb_conv_layers",
        from_rgb_conv_layers
    )

    # Build fromRGB 1x1 conv layers internals through call.
    from_rgb_conv_tensors = build_discriminator_from_rgb_layers(
        from_rgb_conv_layers=from_rgb_conv_layers, params=params
    )
    print_obj(
        "discriminator_network",
        "from_rgb_conv_tensors",
        from_rgb_conv_tensors
    )

    with tf.control_dependencies(control_inputs=from_rgb_conv_tensors):
        # Create empty list to hold discriminator convolutional layer blocks.
        block_layers = []
        block_tensors = []

        # Create base convolutional block's layers, for post-growth.
        block_layers.append(
            create_discriminator_base_conv_layer_block(
                regularizer=regularizer, params=params
            )
        )

        # Create base convolutional block's layer internals using call.
        block_tensors.append(
            build_discriminator_base_conv_layer_block(
                base_conv_layers=block_layers[0], params=params
            )
        )

        # Create growth layer blocks.
        for block_idx in range(
           len(params["discriminator_growth_conv_blocks"])):
            block_layers.append(
                create_discriminator_growth_layer_block(
                    regularizer=regularizer,
                    params=params,
                    block_idx=block_idx
                )
            )
        print_obj("discriminator_network", "block_layers", block_layers)

        # Build growth layer block internals through call.
        for block_idx in range(
           len(params["discriminator_growth_conv_blocks"])):
            block_tensors.append(
                build_discriminator_growth_layer_block(
                    conv_layers=block_layers[block_idx + 1],
                    params=params,
                    block_idx=block_idx
                )
            )

        # Flatten block tensor lists of lists into list.
        block_tensors = [item for sublist in block_tensors for item in sublist]
        print_obj("discriminator_network", "block_tensors", block_tensors)

        with tf.control_dependencies(control_inputs=block_tensors):
            # Create list of transition downsample layers.
            transition_downsample_layers = (
                create_discriminator_growth_transition_downsample_layers(
                    params=params
                )
            )
            print_obj(
                "discriminator_network",
                "transition_downsample_layers",
                transition_downsample_layers
            )

            # Create flatten and logits layers.
            flatten_layer, logits_layer = create_discriminator_logits_layer(
                regularizer=regularizer
            )

            # Build logits layer internals using call.
            logits_tensor = build_discriminator_logits_layer(
                flatten_layer=flatten_layer,
                logits_layer=logits_layer,
                params=params
            )

            with tf.control_dependencies(control_inputs=[logits_tensor]):
                # Get discriminator's logits output tensor.
                train_steps = params["train_steps"]
                num_steps_until_growth = params["num_steps_until_growth"]
                num_stages = train_steps // num_steps_until_growth
                if (num_stages <= 0 or len(params["conv_num_filters"]) == 1):
                    print(
                        "\ndiscriminator_network: NOT GOING TO GROW, SKIP SWITCH CASE!"
                    )
                    # If never going to grow, no sense using the switch case.
                    # 4x4
                    logits = create_base_discriminator_network(
                        X=X,
                        from_rgb_conv_layers=from_rgb_conv_layers,
                        blocks=block_layers,
                        flatten_layer=flatten_layer,
                        logits_layer=logits_layer,
                        params=params
                    )
                else:
                    # Find index based on global step and growth frequency.
                    growth_index = tf.cast(
                        x=tf.floordiv(
                            x=tf.train.get_or_create_global_step(),
                            y=params["num_steps_until_growth"],
                            name="discriminator_global_step_floordiv"
                        ),
                        dtype=tf.int32,
                        name="discriminator_growth_index"
                    )

                    # Switch to case based on number of steps to get logits.
                    logits = tf.switch_case(
                        branch_index=growth_index,
                        branch_fns=[
                            # 4x4
                            lambda: create_base_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                params=params
                            ),
                            # 8x8
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=0
                            ),
                            # 16x16
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=1
                            ),
                            # 32x32
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=2
                            ),
                            # 64x64
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=3
                            ),
                            # 128x128
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=4
                            ),
                            # 256x256
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=5
                            ),
                            # 512x512
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=6
                            ),
                            # 1024x1024
                            lambda: create_growth_transition_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                transition_downsample_layers=transition_downsample_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                alpha_var=alpha_var,
                                params=params,
                                trans_idx=7
                            ),
                            # 1024x1024
                            lambda: create_final_discriminator_network(
                                X=X,
                                from_rgb_conv_layers=from_rgb_conv_layers,
                                blocks=block_layers,
                                flatten_layer=flatten_layer,
                                logits_layer=logits_layer,
                                params=params
                            )
                        ],
                        name="discriminator_switch_case_logits"
                    )

    return logits


def get_discriminator_loss(fake_logits, real_logits, params):
    """Gets discriminator loss.

    Args:
        fake_logits: tensor, shape of
            [cur_batch_size, height * width * depth].
        real_logits: tensor, shape of
            [cur_batch_size, height * width * depth].
        params: dict, user passed parameters.

    Returns:
        Tensor of discriminator's total loss of shape [].
    """
    # Calculate base discriminator loss.
    discriminator_real_loss = tf.reduce_mean(
        input_tensor=real_logits,
        name="discriminator_real_loss"
    )
    print_obj(
        "\nget_discriminator_loss",
        "discriminator_real_loss",
        discriminator_real_loss
    )

    discriminator_generated_loss = tf.reduce_mean(
        input_tensor=fake_logits,
        name="discriminator_generated_loss"
    )
    print_obj(
        "get_discriminator_loss",
        "discriminator_generated_loss",
        discriminator_generated_loss
    )

    discriminator_loss = tf.add(
        x=discriminator_real_loss, y=-discriminator_generated_loss,
        name="discriminator_loss"
    )
    print_obj(
        "get_discriminator_loss",
        "discriminator_loss",
        discriminator_loss
    )

    # Get discriminator gradient penalty loss.
    discriminator_gradients = tf.gradients(
        ys=discriminator_loss,
        xs=tf.trainable_variables(scope="discriminator"),
        name="discriminator_gradients_for_penalty"
    )

    discriminator_gradient_penalty = tf.square(
        x=tf.multiply(
            x=params["discriminator_gradient_penalty_coefficient"],
            y=tf.linalg.global_norm(
                t_list=discriminator_gradients,
                name="discriminator_gradients_global_norm"
            ) - 1.0
        ),
        name="discriminator_gradient_penalty"
    )

    # Get discriminator Wasserstein GP loss.
    discriminator_wasserstein_gp_loss = tf.add(
        x=discriminator_loss,
        y=discriminator_gradient_penalty,
        name="discriminator_wasserstein_gp_loss"
    )

    # Get discriminator regularization losses.
    discriminator_reg_loss = regularization.get_regularization_loss(
        params=params, scope="discriminator"
    )
    print_obj(
        "get_discriminator_loss",
        "discriminator_reg_loss",
        discriminator_reg_loss
    )

    # Combine losses for total losses.
    discriminator_total_loss = tf.math.add(
        x=discriminator_wasserstein_gp_loss,
        y=discriminator_reg_loss,
        name="discriminator_total_loss"
    )
    print_obj(
        "get_discriminator_loss",
        "discriminator_total_loss",
        discriminator_total_loss
    )

    return discriminator_total_loss


## regularization.py

In [ ]:
def get_regularization_loss(params, scope=None):
    """Gets regularization losses from variables attached to a regularizer.

    Args:
        params: dict, user passed parameters.
        scope: str, the name of the variable scope.

    Returns:
        Scalar regularization loss tensor.
    """
    def sum_nd_tensor_list_to_scalar_tensor(t_list):
        """Sums different shape tensors into a scalar tensor.

        Args:
            t_list: list, tensors of varying shapes.

        Returns:
            Scalar tensor.
        """
        # Sum list of tensors into a list of scalars.
        t_reduce_sum_list = [
            tf.reduce_sum(
                # Remove the :0 from the end of the name.
                input_tensor=t, name="{}_reduce_sum".format(t.name[:-2])
            )
            for t in t_list
        ]
        print_obj(
            "\nsum_nd_tensor_list_to_scalar_tensor",
            "t_reduce_sum_list",
            t_reduce_sum_list
        )

        # Add all scalars together into one scalar.
        t_scalar_sum_tensor = tf.add_n(
            inputs=t_reduce_sum_list,
            name="{}_t_scalar_sum_tensor".format(scope)
        )
        print_obj(
            "sum_nd_tensor_list_to_scalar_tensor",
            "t_scalar_sum_tensor",
            t_scalar_sum_tensor
        )

        return t_scalar_sum_tensor

    print_obj("\nget_regularization_loss", "scope", scope)
    lambda1 = params["discriminator_l1_regularization_scale"]
    lambda2 = params["discriminator_l2_regularization_scale"]
    if lambda1 <= 0. and lambda2 <= 0.:
        # No regularization so return zero.
        return tf.zeros(shape=[], dtype=tf.float32)

    # Get list of trainable variables with a regularizer attached in scope.
    trainable_reg_vars_list = tf.get_collection(
        tf.GraphKeys.REGULARIZATION_LOSSES, scope=scope)
    print_obj(
        "get_regularization_loss",
        "trainable_reg_vars_list",
        trainable_reg_vars_list
    )
    
    for var in trainable_reg_vars_list:
        print_obj(
            "get_regularization_loss_{}".format(scope),
            "{}".format(var.name),
            var.graph
        )

    l1_loss = 0.
    if lambda1 > 0.:
        # For L1 regularization, take the absolute value element-wise of each.
        trainable_reg_vars_abs_list = [
            tf.abs(
                x=var,
                # Clean up regularizer scopes in variable names.
                name="{}_abs".format(("/").join(var.name.split("/")[0:3]))
            )
            for var in trainable_reg_vars_list
        ]

        # Get L1 loss
        l1_loss = tf.multiply(
            x=lambda1,
            y=sum_nd_tensor_list_to_scalar_tensor(
                t_list=trainable_reg_vars_abs_list
            ),
            name="{}_l1_loss".format(scope)
        )

    l2_loss = 0.
    if lambda2 > 0.:
        # For L2 regularization, square all variables element-wise.
        trainable_reg_vars_squared_list = [
            tf.square(
                x=var,
                # Clean up regularizer scopes in variable names.
                name="{}_squared".format(("/").join(var.name.split("/")[0:3]))
            )
            for var in trainable_reg_vars_list
        ]
        print_obj(
            "get_regularization_loss",
            "trainable_reg_vars_squared_list",
            trainable_reg_vars_squared_list
        )

        # Get L2 loss
        l2_loss = tf.multiply(
            x=lambda2,
            y=sum_nd_tensor_list_to_scalar_tensor(
                t_list=trainable_reg_vars_squared_list
            ),
            name="{}_l2_loss".format(scope)
        )

    l1_l2_loss = tf.add(
        x=l1_loss, y=l2_loss, name="{}_l1_l2_loss".format(scope)
    )

    return l1_l2_loss


## pgan.py

In [ ]:
def train_network(loss, global_step, alpha_var, params, scope):
    """Trains network and returns loss and train op.

    Args:
        loss: tensor, shape of [].
        global_step: tensor, the current training step or batch in the
            training loop.
        alpha_var: variable, alpha for weighted sum of fade-in of layers.
        params: dict, user passed parameters.
        scope: str, the variables that to train.

    Returns:
        Loss tensor and training op.
    """
    # Create optimizer map.
    optimizers = {
        "Adam": tf.train.AdamOptimizer,
        "Adadelta": tf.train.AdadeltaOptimizer,
        "AdagradDA": tf.train.AdagradDAOptimizer,
        "Adagrad": tf.train.AdagradOptimizer,
        "Ftrl": tf.train.FtrlOptimizer,
        "GradientDescent": tf.train.GradientDescentOptimizer,
        "Momentum": tf.train.MomentumOptimizer,
        "ProximalAdagrad": tf.train.ProximalAdagradOptimizer,
        "ProximalGradientDescent": tf.train.ProximalGradientDescentOptimizer,
        "RMSProp": tf.train.RMSPropOptimizer
    }

    # Get gradients.
    gradients = tf.gradients(
        ys=loss,
        xs=tf.trainable_variables(scope=scope),
        name="{}_gradients".format(scope)
    )

    # Clip gradients.
    if params["{}_clip_gradients".format(scope)]:
        gradients, _ = tf.clip_by_global_norm(
            t_list=gradients,
            clip_norm=params["{}_clip_gradients".format(scope)],
            name="{}_clip_by_global_norm_gradients".format(scope)
        )

    # Zip back together gradients and variables.
    grads_and_vars = zip(gradients, tf.trainable_variables(scope=scope))

    # Get optimizer and instantiate it.
    optimizer = optimizers[params["{}_optimizer".format(scope)]](
        learning_rate=params["{}_learning_rate".format(scope)]
    )

    # Create train op by applying gradients to variables and incrementing
    # global step.
    train_op = optimizer.apply_gradients(
        grads_and_vars=grads_and_vars,
        global_step=global_step,
        name="{}_apply_gradients".format(scope)
    )

    # Update alpha variable to linearly scale from 0 to 1 based on steps.
    alpha_var_update_op = tf.assign(
        ref=alpha_var,
        value=tf.divide(
            x=tf.cast(
                x=tf.mod(x=global_step, y=params["num_steps_until_growth"]),
                dtype=tf.float32
            ),
            y=params["num_steps_until_growth"]
        )
    )

    # Ensure alpha variable gets updated.
    with tf.control_dependencies(control_inputs=[alpha_var_update_op]):
        loss = tf.identity(input=loss, name="train_network_loss_identity")

    return loss, train_op


def resize_real_image(block_idx, image, params):
    """Resizes real images to match the GAN's current size.

    Args:
        block_idx: int, index of current block.
        image: tensor, original image.
        params: dict, user passed parameters.

    Returns:
        Resized image tensor.
    """
    print_obj("\nresize_real_image", "block_idx", block_idx)
    print_obj("resize_real_image", "image", image)

    # Resize image to match GAN size at current block index.
    resized_image = tf.image.resize(
        images=image,
        size=[
            params["generator_projection_dims"][0] * (2 ** block_idx),
            params["generator_projection_dims"][1] * (2 ** block_idx)
        ],
        method="nearest",
        name="resize_real_images_resized_image_{}".format(block_idx)
    )
    print_obj("resize_real_images", "resized_image", resized_image)

    return resized_image


def resize_real_images(image, params):
    """Resizes real images to match the GAN's current size.

    Args:
        image: tensor, original image.
        params: dict, user passed parameters.

    Returns:
        Resized image tensor.
    """
    print_obj("\nresize_real_images", "image", image)
    # Resize real image for each block.
    num_stages = params["train_steps"] // params["num_steps_until_growth"]
    if (num_stages <= 0 or len(params["conv_num_filters"]) == 1):
        print(
            "\nresize_real_images: NEVER GOING TO GROW, SKIP SWITCH CASE!"
        )
        # If we never are going to grow, no sense using the switch case.
        resized_image = resize_real_image(0, image, params)  # 4x4
    else:
        # Find growth index based on global step and growth frequency.
        growth_index = tf.cast(
            x=tf.floordiv(
                x=tf.train.get_or_create_global_step(),
                y=params["num_steps_until_growth"],
                name="resize_real_images_global_step_floordiv"
            ),
            dtype=tf.int32,
            name="resize_real_images_growth_index"
        )

        # Switch to case based on number of steps for resized image.
        resized_image = tf.switch_case(
            branch_index=growth_index,
            branch_fns=[
                lambda: resize_real_image(0, image, params),  # 4x4
                lambda: resize_real_image(1, image, params),  # 8x8
                lambda: resize_real_image(2, image, params),  # 16x16
                lambda: resize_real_image(3, image, params),  # 32x32
                lambda: resize_real_image(4, image, params),  # 64x64
                lambda: resize_real_image(5, image, params),  # 128x128
                lambda: resize_real_image(6, image, params),  # 256x256
                lambda: resize_real_image(7, image, params),  # 512x512
                lambda: resize_real_image(8, image, params),  # 1024x1024
            ],
            name="resize_real_images_switch_case_resized_image"
        )
        print_obj(
            "resize_real_images", "selected resized_image", resized_image
        )

    return resized_image


def pgan_model(features, labels, mode, params):
    """Progressively Growing GAN custom Estimator model function.

    Args:
        features: dict, keys are feature names and values are feature tensors.
        labels: tensor, label data.
        mode: tf.estimator.ModeKeys with values of either TRAIN, EVAL, or
            PREDICT.
        params: dict, user passed parameters.

    Returns:
        Instance of `tf.estimator.EstimatorSpec` class.
    """
    print_obj("\npgan_model", "features", features)
    print_obj("pgan_model", "labels", labels)
    print_obj("pgan_model", "mode", mode)
    print_obj("pgan_model", "params", params)

    # Loss function, training/eval ops, etc.
    predictions_dict = None
    loss = None
    train_op = None
    eval_metric_ops = None
    export_outputs = None

    # Create alpha variable to use for weighted sum for smooth fade-in.
    alpha_var = tf.get_variable(
        name="alpha_var",
        dtype=tf.float32,
        initializer=tf.zeros(shape=[], dtype=tf.float32),
        trainable=False
    )
    print_obj("pgan_model", "alpha_var", alpha_var)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # Extract given latent vectors from features dictionary.
        Z = tf.cast(x=features["Z"], dtype=tf.float32)

        # Get predictions from generator.
        generated_images = generator.generator_network(
            Z=Z, alpha_var=alpha_var, params=params
        )

        # Create predictions dictionary.
        predictions_dict = {
            "generated_images": generated_images
        }

        # Create export outputs.
        export_outputs = {
            "predict_export_outputs": tf.estimator.export.PredictOutput(
                outputs=predictions_dict)
        }
    else:
        # Extract image from features dictionary.
        X = features["image"]

        # Get dynamic batch size in case of partial batch.
        cur_batch_size = tf.shape(
            input=X,
            out_type=tf.int32,
            name="pgan_model_cur_batch_size"
        )[0]

        # Create random noise latent vector for each batch example.
        Z = tf.random.normal(
            shape=[cur_batch_size, params["latent_size"]],
            mean=0.0,
            stddev=1.0,
            dtype=tf.float32
        )

        # Get generated image from generator network from gaussian noise.
        print("\nCall generator with Z = {}.".format(Z))
        generator_outputs = generator.generator_network(
            Z=Z, alpha_var=alpha_var, params=params
        )

        # Get fake logits from discriminator using generator's output image.
        print("\nCall discriminator with generator_outputs = {}.".format(
            generator_outputs
        ))

        fake_logits = discriminator.discriminator_network(
            X=generator_outputs, alpha_var=alpha_var, params=params
        )
        
        # Resize real images based on the current size of the GAN.
        real_image = resize_real_images(X, params)

        # Get real logits from discriminator using real image.
        print("\nCall discriminator with real_image = {}.".format(
            real_image
        ))

        real_logits = discriminator.discriminator_network(
            X=real_image, alpha_var=alpha_var, params=params
        )

        # Get generator total loss.
        generator_total_loss = generator.get_generator_loss(
            fake_logits=fake_logits, params=params
        )

        # Get discriminator total loss.
        discriminator_total_loss = discriminator.get_discriminator_loss(
            fake_logits=fake_logits, real_logits=real_logits, params=params
        )

        if mode == tf.estimator.ModeKeys.TRAIN:
            # Get global step.
            global_step = tf.train.get_or_create_global_step()

            # Determine if it is time to train generator or discriminator.
            cycle_step = tf.mod(
                x=global_step,
                y=tf.cast(
                    x=tf.add(
                        x=params["generator_train_steps"],
                        y=params["discriminator_train_steps"]
                    ),
                    dtype=tf.int64
                ),
                name="pgan_model_cycle_step"
            )

            # Create choose generator condition.
            condition = tf.less(
                x=cycle_step, y=params["generator_train_steps"]
            )

            # Needed for batch normalization, but has no effect otherwise.
            update_ops = tf.get_collection(key=tf.GraphKeys.UPDATE_OPS)

            with tf.control_dependencies(control_inputs=update_ops):
                # Conditionally choose to train generator or discriminator.
                loss, train_op = tf.cond(
                    pred=condition,
                    true_fn=lambda: train_network(
                        loss=generator_total_loss,
                        global_step=global_step,
                        alpha_var=alpha_var,
                        params=params,
                        scope="generator"
                    ),
                    false_fn=lambda: train_network(
                        loss=discriminator_total_loss,
                        global_step=global_step,
                        alpha_var=alpha_var,
                        params=params,
                        scope="discriminator"
                    )
                )
        else:
            loss = discriminator_total_loss

            # Concatenate discriminator logits and labels.
            discriminator_logits = tf.concat(
                values=[real_logits, fake_logits],
                axis=0,
                name="discriminator_concat_logits"
            )

            discriminator_labels = tf.concat(
                values=[
                    tf.ones_like(tensor=real_logits),
                    tf.zeros_like(tensor=fake_logits)
                ],
                axis=0,
                name="discriminator_concat_labels"
            )

            # Calculate discriminator probabilities.
            discriminator_probabilities = tf.nn.sigmoid(
                x=discriminator_logits, name="discriminator_probabilities"
            )

            # Create eval metric ops dictionary.
            eval_metric_ops = {
                "accuracy": tf.metrics.accuracy(
                    labels=discriminator_labels,
                    predictions=discriminator_probabilities,
                    name="pgan_model_accuracy"
                ),
                "precision": tf.metrics.precision(
                    labels=discriminator_labels,
                    predictions=discriminator_probabilities,
                    name="pgan_model_precision"
                ),
                "recall": tf.metrics.recall(
                    labels=discriminator_labels,
                    predictions=discriminator_probabilities,
                    name="pgan_model_recall"
                ),
                "auc_roc": tf.metrics.auc(
                    labels=discriminator_labels,
                    predictions=discriminator_probabilities,
                    num_thresholds=200,
                    curve="ROC",
                    name="pgan_model_auc_roc"
                ),
                "auc_pr": tf.metrics.auc(
                    labels=discriminator_labels,
                    predictions=discriminator_probabilities,
                    num_thresholds=200,
                    curve="PR",
                    name="pgan_model_auc_pr"
                )
            }

    # Return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs=export_outputs
    )


## serving.py

In [ ]:
def serving_input_fn(params):
    """Serving input function.

    Args:
        params: dict, user passed parameters.

    Returns:
        ServingInputReceiver object containing features and receiver tensors.
    """
    # Create placeholders to accept data sent to the model at serving time.
    # shape = (batch_size,)
    feature_placeholders = {
        "Z": tf.placeholder(
            dtype=tf.float32,
            shape=[None, params["latent_size"]],
            name="serving_input_placeholder_Z"
        )
    }

    print_obj(
        "\nserving_input_fn",
        "feature_placeholders",
        feature_placeholders
    )

    # Create clones of the feature placeholder tensors so that the SavedModel
    # SignatureDef will point to the placeholder.
    features = {
        key: tf.identity(
            input=value,
            name="serving_input_fn_identity_placeholder_{}".format(key)
        )
        for key, value in feature_placeholders.items()
    }

    print_obj(
        "serving_input_fn",
        "features",
        features
    )

    return tf.estimator.export.ServingInputReceiver(
        features=features, receiver_tensors=feature_placeholders
    )


## model.py

In [ ]:
def train_and_evaluate(args):
    """Trains and evaluates custom Estimator model.

    Args:
        args: dict, user passed parameters.

    Returns:
        `Estimator` object.
    """
    # Set logging to be level of INFO.
    tf.logging.set_verbosity(tf.logging.INFO)

    # Create our custom estimator using our model function.
    estimator = tf.estimator.Estimator(
        model_fn=pgan.pgan_model,
        model_dir=args["output_dir"],
        params=args
    )

    # Create train spec to read in our training data.
    train_spec = tf.estimator.TrainSpec(
        input_fn=input.read_dataset(
            filename=args["train_file_pattern"],
            mode=tf.estimator.ModeKeys.TRAIN,
            batch_size=args["train_batch_size"],
            params=args
        ),
        max_steps=args["train_steps"]
    )

    # Create exporter to save out the complete model to disk.
    exporter = tf.estimator.LatestExporter(
        name="exporter",
        serving_input_receiver_fn=lambda: serving.serving_input_fn(args)
    )

    # Create eval spec to read in our validation data and export our model.
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input.read_dataset(
            filename=args["eval_file_pattern"],
            mode=tf.estimator.ModeKeys.EVAL,
            batch_size=args["eval_batch_size"],
            params=args
        ),
        steps=args["eval_steps"],
        start_delay_secs=args["start_delay_secs"],
        throttle_secs=args["throttle_secs"],
        exporters=exporter
    )

    # Create train and evaluate loop to train and evaluate our estimator.
    tf.estimator.train_and_evaluate(
        estimator=estimator, train_spec=train_spec, eval_spec=eval_spec)


## Run model

In [ ]:
shutil.rmtree(path=arguments["output_dir"], ignore_errors=True)
estimator = train_and_evaluate(arguments)

## Prediction

In [ ]:
!ls trained_model/export/exporter

In [ ]:
# predict_fn = tf.contrib.predictor.from_saved_model(
#     "trained_model/export/exporter/1590991040"
# )
# predictions = predict_fn(
#     {
#         "Z": np.random.normal(size=(500, 512))
#     }
# )

Convert image back to the original scale.

In [ ]:
# generated_images = np.clip(
#     a=((predictions["generated_images"] + 1.0) * (255. / 2)).astype(np.int32),
#     a_min=0,
#     a_max=255
# )

In [ ]:
# print(generated_images.shape)

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for i in range(5):
#     plt.subplot(1, 5, i + 1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(generated_images[i], cmap=plt.cm.binary)
# plt.show()